# 입문자를 위한, 파이썬/R 데이터 분석

## Part(2) - Day12  : 

]

## Today's mission

- [ 군집(CLUSTERING) ] : 머신러닝 데이터 분석 예제 실습
1. k-Means clustering
2. DBSCAN(Densitry-Based Spatial Clestring of Applications with Noise)

]

# [ 군집(CLUSTERING) ]

## 1. k-Means clustering

In [ ]:
# -*- coding: utf-8 -*-
### 기본 라이브러리 불러오기
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
'''
[Step 1] 데이터 준비
'''
# Wholesale customers 데이터셋 가져오기 (출처: UCI ML Repository)
uci_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/\
00292/Wholesale%20customers%20data.csv'
df = pd.read_csv(uci_path, header=0)
df

In [ ]:
uci_path

In [ ]:
'''
[Step 2] 데이터 탐색
'''
# 데이터 살펴보기
df.head()

In [ ]:
# 데이터 자료형 확인
df.info()

In [ ]:
# 데이터 통계 요약정보 확인
df.describe()

In [ ]:
'''
[Step 3] 데이터 전처리
'''
# 분석에 사용할 속성을 선택
X = df.iloc[:, :]
X[:5]

In [ ]:
# 설명 변수 데이터를 정규화
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
print(X[:5])

In [ ]:
'''
[Step 4] k-means 군집 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 cluster 군집 모형 가져오기
from sklearn import cluster

# 모형 객체 생성 
# 모형은 8개의 속성(변수)를 이용하여 각 관측값을 5개의 클러스터로 구분
kmeans = cluster.KMeans(init='k-means++', n_clusters=5, n_init=10)

In [ ]:
# 모형 학습
kmeans.fit(X)   

In [ ]:
# 예측 (군집) 
cluster_label = kmeans.labels_   
print(cluster_label)

In [ ]:
# 예측 결과를 데이터프레임에 추가
df['Cluster'] = cluster_label
print(df.head())

In [ ]:
df[df['Cluster']==1].describe()

In [ ]:
# 그래프로 표현 - 시각화
df.plot(kind='scatter', x='Grocery', y='Frozen', c='Cluster', cmap='Set1', 
        colorbar=True, figsize=(10, 10))

In [ ]:
df.plot(kind='scatter', x='Milk', y='Delicassen', c='Cluster', cmap='Set1', 
        colorbar=True, figsize=(10, 10))

In [ ]:
# 다른 값들에 비해 지나치게큰 값으로 구성된 클러스터(0, 4)를 제외 - 값이 몰려 있는 구간을 자세하게 분석
mask = (df['Cluster'] == 0) | (df['Cluster'] == 4)
ndf = df[~mask]

In [ ]:
ndf.plot(kind='scatter', x='Grocery', y='Frozen', c='Cluster', cmap='Set1', 
        colorbar=True, figsize=(10, 10))

In [ ]:
ndf.plot(kind='scatter', x='Milk', y='Delicassen', c='Cluster', cmap='Set1', 
        colorbar=True, figsize=(10, 10))

In [ ]:
# Cluster별 분석
df['Cluster'].unique()

In [ ]:
df[((df['Cluster'] == 0) | (df['Cluster'] == 4))].describe()

In [ ]:
df[df['Cluster'].isin([1,2,3])].describe()

### 2. DBSCAN(Densitry-Based Spatial Clestring of Applications with Noise)

In [ ]:
#!pip install folium

In [ ]:
# -*- coding: utf-8 -*-
### 기본 라이브러리 불러오기
import pandas as pd
import folium

In [ ]:
'''
[Step 1] 데이터 준비/ 기본 설정
'''
pd.set_option('display.max_columns', 100)
# 서울시내 중학교 진학률 데이터셋 (출처: 교육???)
file_path = 'data/2016_middle_shcool_graduates_report.xlsx'
df = pd.read_excel(file_path, header=0)
df

In [ ]:
# IPython Console 디스플레이 옵션 설정하기
pd.set_option('display.width', None)        # 출력화면의 너비
pd.set_option('display.max_rows', 100)      # 출력할 행의 개수 한도
pd.set_option('display.max_columns', 100)    # 출력할 열의 개수 한도
pd.set_option('display.max_colwidth', 20)   # 출력할 열의 너비
pd.set_option('display.unicode.east_asian_width', True)   # 유니코드 사용 너비 조정
# 열 이름 배열을 출력
df.columns

In [ ]:
'''
[Step 2] 데이터 탐색
'''
# 데이터 살펴보기
df.head()

In [ ]:
# 데이터 자료형 확인
df.info()

In [ ]:
# 데이터 통계 요약정보 확인
df.describe()

In [ ]:
# 지도에 위치 표시
mschool_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)
# 중학교 위치정보를 CircleMarker로 표시
for name, lat, lng in zip(df.학교명, df.위도, df.경도):
    folium.CircleMarker([lat, lng],
                        radius=5,              # 원의 반지름
                        color='brown',         # 원의 둘레 색상
                        fill=True,
                        fill_color='coral',    # 원을 채우는 색
                        fill_opacity=0.7,      # 투명도    
                        popup=name
    ).add_to(mschool_map)
# 지도를 html 파일로 저장하기
mschool_map.save('data/seoul_mschool_location.html')

In [ ]:
'''
[Step 3] 데이터 전처리
'''
# 원핫인코딩(더미 변수)
from sklearn import preprocessing    

label_encoder = preprocessing.LabelEncoder()     # label encoder 생성
onehot_encoder = preprocessing.OneHotEncoder()   # one hot encoder 생성

onehot_location = label_encoder.fit_transform(df['지역'])
onehot_code = label_encoder.fit_transform(df['코드'])
onehot_type = label_encoder.fit_transform(df['유형'])
onehot_day = label_encoder.fit_transform(df['주야'])

df['location'] = onehot_location
df['code'] = onehot_code
df['type'] = onehot_type
df['day'] = onehot_day

df.head(1)

In [ ]:
df['location'].unique()

In [ ]:
'''
[Step 4] DBSCAN 군집 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 cluster 군집 모형 가져오기 
from sklearn import cluster

# 분석에 사용할 속성을 선택 (과학고, 외고국제고, 자사고 진학률)
columns_list = [10, 11, 14]
X = df.iloc[:, columns_list]
print(X[:5])

In [ ]:
# 설명 변수 데이터를 정규화
X = preprocessing.StandardScaler().fit(X).transform(X)
X

In [ ]:
# DBSCAN 모형 객체 생성
dbm = cluster.DBSCAN(eps=0.2, min_samples=5)

In [ ]:
# 모형 학습
dbm.fit(X) 

In [ ]:
# 예측 (군집) 
cluster_label = dbm.labels_   
print(cluster_label)

In [ ]:
pd.DataFrame(cluster_label).describe()

In [ ]:
# 예측 결과를 데이터프레임에 추가
df['Cluster'] = cluster_label
print(df.head())

In [ ]:
# 클러스터 값으로 그룹화하고, 그룹별로 내용 출력 (첫 5행만 출력)
grouped_cols = [0, 1, 3] + columns_list
grouped = df.groupby('Cluster')
for key, group in grouped:
    print('* key :', key)
    print('* number :', len(group))    
    print(group.iloc[:, grouped_cols].head())
    print('\n')

In [ ]:
# 그래프로 표현 - 시각화
colors = {-1:'gray', 0:'coral', 1:'blue', 2:'green', 3:'red', 4:'purple', 
          5:'orange', 6:'brown', 7:'brick', 8:'yellow', 9:'magenta', 10:'cyan', 11:'black'}

cluster_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

for name, lat, lng, clus in zip(df.학교명, df.위도, df.경도, df.Cluster):  
    folium.CircleMarker([lat, lng],
                        radius=5,                   # 원의 반지름
                        color=colors[clus],         # 원의 둘레 색상
                        fill=True,
                        fill_color=colors[clus],    # 원을 채우는 색
                        fill_opacity=0.7,           # 투명도    
                        popup=name
    ).add_to(cluster_map)

In [ ]:
# 지도를 html 파일로 저장하기
cluster_map.save('data/seoul_mschool_cluster.html')

In [ ]:
# X2 데이터셋에 대하여 위의 과정을 반복(과학고, 외고국제고, 자사고 진학률 + 유형)
columns_list2 = [10, 11, 14, 22]
X2 = df.iloc[:, columns_list2]
print(X2[:5])
print('\n')

In [ ]:
X2 = preprocessing.StandardScaler().fit(X2).transform(X2)
dbm2 = cluster.DBSCAN(eps=0.2, min_samples=5)
dbm2.fit(X2)  
df['Cluster2'] = dbm2.labels_   

In [ ]:
grouped2_cols = [0, 1, 3] + columns_list2
grouped2 = df.groupby('Cluster2')
for key, group in grouped2:
    print('* key :', key)
    print('* number :', len(group))    
    print(group.iloc[:, grouped2_cols].head())
    print('\n')

In [ ]:
cluster2_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

In [ ]:
for name, lat, lng, clus in zip(df.학교명, df.위도, df.경도, df.Cluster2):  
    folium.CircleMarker([lat, lng],
                        radius=5,                   # 원의 반지름
                        color=colors[clus],         # 원의 둘레 색상
                        fill=True,
                        fill_color=colors[clus],    # 원을 채우는 색
                        fill_opacity=0.7,           # 투명도    
                        popup=name
    ).add_to(cluster2_map)

In [ ]:
# 지도를 html 파일로 저장하기
cluster2_map.save('data/seoul_mschool_cluster2.html')

In [ ]:
# X3 데이터셋에 대하여 위의 과정을 반복(과학고, 외고_국제고)
columns_list3 = [10, 11]
X3 = df.iloc[:, columns_list3]
print(X3[:5])
print('\n')

In [ ]:
X3 = preprocessing.StandardScaler().fit(X3).transform(X3)
dbm3 = cluster.DBSCAN(eps=0.2, min_samples=5)
dbm3.fit(X3)  
df['Cluster3'] = dbm3.labels_ 

In [ ]:
grouped3_cols = [0, 1, 3] + columns_list3
grouped3 = df.groupby('Cluster3')
for key, group in grouped3:
    print('* key :', key)
    print('* number :', len(group))    
    print(group.iloc[:, grouped3_cols].head())
    print('\n')

In [ ]:
cluster3_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

In [ ]:
for name, lat, lng, clus in zip(df.학교명, df.위도, df.경도, df.Cluster3):  
    folium.CircleMarker([lat, lng],
                        radius=5,                   # 원의 반지름
                        color=colors[clus],         # 원의 둘레 색상
                        fill=True,
                        fill_color=colors[clus],    # 원을 채우는 색
                        fill_opacity=0.7,           # 투명도    
                        popup=name
    ).add_to(cluster3_map)

In [ ]:
# 지도를 html 파일로 저장하기
cluster3_map.save('data/seoul_mschool_cluster3.html')